In [0]:
import tensorflow as tf
import pandas as pd
import tensorflow.feature_column as fc
import numpy as np
# from sklearn import test_train_split
# write an input fn as we have already written

#define the feature columns
data = pd.read_csv('algebra.csv')
x1_norm_factor=data['Input1'].max()-data['Input1'].min()
x2_norm_factor=data['Input2'].max()-data['Input2'].min()
y1_norm_factor=data['Output1'].max()-data['Output1'].min()
y2_norm_factor=data['Output2'].max()-data['Output2'].min()
BATCH_SIZE = 128
NUM_EPOCHS = 100

    

In [0]:
def get_input_3():
    global data
    data = data/(data.max()-data.min())
    train_x1 = data['Input1'].values
    train_x2 = data['Input2'].values
    train_y1 = pd.to_numeric(data['Output1'].values, downcast = 'float')
    dataset = tf.data.Dataset.from_tensor_slices(({'Feature1':train_x1,'Feature2':train_x2},train_y1))
    return dataset.batch(BATCH_SIZE).repeat(NUM_EPOCHS)

def get_input_4():
    global data
    data = data/(data.max()-data.min())
    train_x1 = data['Input1'].values
    train_x2 = data['Input2'].values
    train_y2 = pd.to_numeric(data['Output2'].values, downcast = 'float')
    dataset = tf.data.Dataset.from_tensor_slices(({'Feature1':train_x1,'Feature2':train_x2},train_y2))

    return dataset.batch(BATCH_SIZE).repeat(NUM_EPOCHS)
                                                        
def get_input_3_4():
    global data
    data = data/(data.max()-data.min())
    train_x1 = data['Input1'].values
    train_x2 = data['Input2'].values
    train_y1 = pd.to_numeric(data['Output1'].values, downcast = 'float')
    train_y2 = pd.to_numeric(data['Output2'].values, downcast = 'float')

    labels = list(zip(train_y1,train_y2))
    dataset = tf.data.Dataset.from_tensor_slices(({'Feature1':train_x1,'Feature2':train_x2},labels))

    return dataset.batch(BATCH_SIZE).repeat(NUM_EPOCHS)
    

In [0]:
class MyDNNRegressor(tf.estimator.Estimator):  
    def get_layer_activations(self,id):   
        weights,bias=self.get_layer_params(id)
        x=np.array([1.0,1.0])
        out=np.add(np.matmul(weights,x),bias)
        out=tf.convert_to_tensor(out)
        out=tf.nn.relu(out)
        sess = tf.Session()
        with sess.as_default():
            out = out.eval()
            return out
        
        
    def get_layer_params(self,id):
        weights=self.get_variable_value(id+'/kernel')
        bias=self.get_variable_value(id+'/bias')
        return weights,bias

In [0]:
def my_model_3(features, labels, mode, params):
    top = tf.feature_column.input_layer(features, params["feature_columns"])
    for units in params.get("hidden_units", [20]):
        top = tf.layers.dense(inputs=top, units=units, activation=tf.nn.relu,name="mylayer")
    output_layer = tf.layers.dense(inputs=top, units=params["output_units"],name="output_layer")
    predictions = tf.squeeze(output_layer,1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
        mode=mode, predictions={"predictions": predictions})
    average_loss = tf.losses.mean_squared_error(labels, predictions)
    batch_size = tf.shape(labels)[0]
    total_loss = tf.to_float(batch_size) * average_loss
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = params.get("optimizer", tf.train.AdamOptimizer)
        optimizer = optimizer(params.get("learning_rate", None))
        train_op = optimizer.minimize(
            loss=average_loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=total_loss, train_op=train_op)
    assert mode == tf.estimator.ModeKeys.EVAL
    rmse = tf.metrics.root_mean_squared_error(labels, predictions)
    acc  = tf.metrics.accuracy(labels,predictions)
    eval_metrics = {"rmse": rmse,"accuracy":acc}
    return tf.estimator.EstimatorSpec(
        mode=mode,
        loss=total_loss,
        eval_metric_ops=eval_metrics)


In [0]:
def my_model_4(features, labels, mode, params):
    top = tf.feature_column.input_layer(features, params["feature_columns"])
    for units in params.get("hidden_units", [20]):
        top = tf.layers.dense(inputs=top, units=units, activation=tf.nn.relu,name="mylayer")
    output_layer = tf.layers.dense(inputs=top, units=params["output_units"],name="output_layer")
    predictions = tf.squeeze(output_layer,1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
        mode=mode, predictions={"predictions": predictions})
    average_loss = tf.losses.mean_squared_error(labels, predictions)
    batch_size = tf.shape(labels)[0]
    total_loss = tf.to_float(batch_size) * average_loss
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = params.get("optimizer", tf.train.AdamOptimizer)
        optimizer = optimizer(params.get("learning_rate", None))
        train_op = optimizer.minimize(
            loss=average_loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=total_loss, train_op=train_op)
    assert mode == tf.estimator.ModeKeys.EVAL
    rmse = tf.metrics.root_mean_squared_error(labels, predictions)
    acc  = tf.metrics.accuracy(labels,predictions)
    eval_metrics = {"rmse": rmse,"accuracy":acc}
    return tf.estimator.EstimatorSpec(
        mode=mode,
        loss=total_loss,
        eval_metric_ops=eval_metrics)

In [0]:
def my_model_34(features, labels, mode, params):
    top = tf.feature_column.input_layer(features, params["feature_columns"])
    for units in params.get("hidden_units", [20]):
        top = tf.layers.dense(inputs=top, units=units, activation=tf.nn.relu,name="mylayer")
    output_layer = tf.layers.dense(inputs=top, units=params["output_units"],name="output_layer")
    predictions = output_layer
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
        mode=mode, predictions={"predictions": predictions})
    average_loss = tf.losses.mean_squared_error(labels, predictions)
    batch_size = tf.shape(labels)[0]
    total_loss = tf.to_float(batch_size) * average_loss
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = params.get("optimizer", tf.train.AdamOptimizer)
        optimizer = optimizer(params.get("learning_rate", None))
        train_op = optimizer.minimize(
            loss=average_loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=total_loss, train_op=train_op)
    assert mode == tf.estimator.ModeKeys.EVAL
    predictions = output_layer
    rmse = tf.metrics.root_mean_squared_error(labels, predictions)
    acc  = tf.metrics.accuracy(labels,predictions)
    eval_metrics = {"rmse": rmse,"accuracy":acc}
    return tf.estimator.EstimatorSpec(
        mode=mode,
        loss=total_loss,
        eval_metric_ops=eval_metrics)

In [22]:
#the first equation with column 3 as the output and column 1&2 as input
feature_columns = [fc.numeric_column(key='Feature1',dtype=tf.float32),fc.numeric_column(key='Feature2',dtype=tf.float32)]
classifier=MyDNNRegressor(
    model_fn=my_model_4,
    model_dir='./44',
    params={
        'feature_columns': feature_columns,
        'hidden_units': [3],
        'output_units':1,
        "learning_rate": 0.001,
        "optimizer": tf.train.AdamOptimizer
    }
)
classifier.train(input_fn=get_input_4,steps=NUM_EPOCHS)
metrics=classifier.evaluate(input_fn=get_input_4)
print("metrics-col3-output:",metrics)
    

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './44', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2d0f8f8748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./44/model.ckpt.
INFO:tensorflow:loss = 11.980504

In [23]:
hidden_weights,hidden_bias=classifier.get_layer_params('mylayer')
print("Hidden_weights",hidden_weights)
print("Hidden_bias",hidden_bias)

Hidden_weights [[ 0.46781093 -0.8223893   0.8388115 ]
 [-0.72959733  0.11084794 -1.116663  ]]
Hidden_bias [-0.0790143  -0.06482318 -0.07755946]


In [24]:
output_weights,output_bias=classifier.get_layer_params('output_layer')
print("Output_weights",output_weights)
print("Output_bias",output_bias)

Output_weights [[-0.8879254]
 [ 1.1438901]
 [-0.5722948]]
Output_bias [0.06554074]


In [33]:
hidden_activation=classifier.get_layer_activations('mylayer')
print(hidden_activation)

[1.02846759 0.        ]


In [14]:
#the second equation with column 4 as the output and column 1&2 as input
feature_columns = [fc.numeric_column(key='Feature1',dtype=tf.float32),fc.numeric_column(key='Feature2',dtype=tf.float32)]
classifier=MyDNNRegressor(
    model_fn=my_model_3,
    model_dir='./03',
    params={
        'feature_columns': feature_columns,
        'hidden_units': [2],
        'output_units':1,
        "learning_rate": 0.001,
        "optimizer": tf.train.AdamOptimizer
    }
)
classifier.train(input_fn=get_input_3,steps=NUM_EPOCHS)
metrics=classifier.evaluate(input_fn=get_input_3)
print("metrics-col3-output:",metrics)
hidden_weights,hidden_bias=classifier.get_layer_params('mylayer')
print("Hidden_weights",hidden_weights)
print("Hidden_bias",hidden_bias)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './03', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fca789d31d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./03/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoint

In [15]:
output_weights,output_bias=classifier.get_layer_params('output_layer')
print("Output_weights",output_weights)
print("Output_bias",output_bias)

Output_weights [[-1.1394113]
 [ 0.3539226]]
Output_bias [0.0779909]


In [35]:
hidden_activation=classifier2.get_layer_activations('mylayer')
print(hidden_activation)

[1.04871748 0.        ]


In [7]:
#the third problem with column 3&4 as the output and column 1&2 as input
feature_columns = [fc.numeric_column(key='Feature1',dtype=tf.float32),fc.numeric_column(key='Feature2',dtype=tf.float32)]
classifier=MyDNNRegressor(
    model_fn=my_model_34,
    model_dir='./343',
    params={
        'feature_columns': feature_columns,
        # Two hidden layers of 10 nodes each.
        'hidden_units': [3],
        # The model must choose between 3 classes.
        'output_units':2,
        "learning_rate": 0.001,
        "optimizer": tf.train.AdamOptimizer
    }
)
classifier.train(input_fn=get_input_3_4,steps=NUM_EPOCHS)
metrics=classifier.evaluate(input_fn=get_input_3_4)
print("metrics-col343-output:",metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './343', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fca793ce1d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./343/model.ckpt.
INFO:tensorflow:loss = 11.2125

In [9]:
hidden_weights,hidden_bias=classifier.get_layer_params('mylayer')
print("Hidden_weights",hidden_weights)
print("Hidden_bias",hidden_bias)

Hidden_weights [[-0.40600175 -0.8305598   0.7578669 ]
 [-0.6333935   0.4068953  -0.5944415 ]]
Hidden_bias [ 0.         -0.08481121 -0.08135027]


In [10]:
output_weights,output_bias=classifier.get_layer_params('output_layer')
print("Output_weights",output_weights)
print("Output_bias",output_bias)

Output_weights [[ 0.24737692  0.10847652]
 [ 0.66898364 -0.26196858]
 [-0.37944865 -0.63974375]]
Output_bias [0.03890631 0.06793098]


In [58]:
hidden_activation=classifier3.get_layer_activations('mylayer')
print(hidden_activation)

[1.04871748 0.        ]


In [0]:
# zero_inp=tf.estimator.inputs.numpy_input_fn(x = {'Feature1':np.array([0.0],dtype = np.float32),'Feature2':np.array([0.0],dtype = np.float32)}, shuffle = False)
# one_inp = tf.estimator.inputs.numpy_input_fn(x = {'Feature1':np.array([1.0],dtype = np.float32),'Feature2':np.array([1.0],dtype = np.float32)}, shuffle = False)
# bias=list(classifier.predict(zero_inp))[0]['predictions']
# one_inp_with_bias = list(classifier.predict(one_inp))[0]['predictions']
# print("a1, a2 ---> ",[one_inp_with_bias[0]-bias[0],one_inp_with_bias[1]-bias[1]])
# print("b1, b2 ---> ",bias)


In [0]:
#Incomplete!!
def guess_equation(x1,x2):
    inp_1=x1/x1_norm_factor
    inp_2=x2/x2_norm_factor
    ex1=tf.estimator.inputs.numpy_input_fn(x = {'Feature1':np.array([inp_1],dtype = np.float32),'Feature2':np.array([inp2],dtype = np.float32)}, shuffle = False)
    y1=classifier1.predict(ex1)
    
    